**Url**

https://boeing.mediaroom.com/news-releases-statements?l=100&o=200

In [1]:
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd,requests,bs4,re,time,io,pytesseract,easyocr,random
from pdfminer.high_level import extract_text
from pathlib import Path
from pdf2image import convert_from_path
from selenium.webdriver.common.by import By
from goose3 import Goose
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver

reader = easyocr.Reader(['en'])

import warnings
warnings.filterwarnings("ignore")

%autosave 1

D:\Anaconda\envs\gpu_16march2022\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Autosaving every 1 seconds


In [2]:
SITE_NAME='Boeing'

DOMAIN = "https://boeing.mediaroom.com"

SITE_LINK="https://boeing.mediaroom.com/news-releases-statements?l=100&o=0"

In [3]:
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

def remove_esc_chars(text):
    return text.replace("\n", " ").replace("\t", " ").replace("\r", " ")

In [4]:
def get_text(link):
    g = Goose()
    article_extract = g.extract(url=link)
    article = remove_esc_chars(article_extract.cleaned_text)
    meta_data = remove_esc_chars(article_extract.meta_description)
    whole_data = meta_data+article
    text = whole_data.strip()
    
    if len(text) < 10:
        try:
            response = requests.get(link)
            text = remove_esc_chars(extract_text(io.BytesIO(response.content)))

            if len(text) < 10:
                texts = ""
                r = requests.get(link)
                filename = Path('temp.pdf')
                filename.write_bytes(r.content)

                pages = convert_from_path('temp.pdf', 500)
                for x in pages:
                    x.save("temp.jpg")
                    output = reader.readtext("temp.jpg")
                    for o in output:
                        texts += o[1]

                text = remove_esc_chars(texts)
        except:
            text = ""
    
    return text

In [5]:
article_list = []
pagination = 0
last_page = 11000

last_page = 200

while pagination < last_page:
    url = f"https://boeing.mediaroom.com/news-releases-statements?l=100&o={pagination}"
    soup = parse_webpage_bs(url)
    if soup != None:
        elements = soup.findAll('li',{'class' :'wd_item'})
        for element in elements[:10]:
            #title,published_date,link,thumbnail,author
            title = element.find('div',{'class' :'wd_title'}).text.strip()
            published_date = element.find('div',{'class' :'wd_date'}).text.strip()
            link = element.find('div',{'class' :'wd_tooltip-trigger'}).find('a')['href']
            author = SITE_NAME
            
            try:
                thumbnail = DOMAIN + element.find('div',{'class' :'wd_thumbnail'}).find('img')['src']
            except:
                
                thumbnail = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Boeing_full_logo.svg/1920px-Boeing_full_logo.svg.png"
            
            #text 
            text = get_text(link)
            
            article = (published_date.strip(),title.strip(),text.strip(),link.strip(),thumbnail.strip(),author.strip())
            article_list.append(article)
            print(published_date,title)
            
    pagination +=100

Jun 1, 2022 Boeing Teams with Canadian Industry to Offer P-8A Poseidon
Jun 1, 2022 Boeing Statement on Germany’s STH Heavy-Lift Helicopter Program
May 25, 2022 Boeing, NASA Complete First Starliner Space Station Flight Test
May 25, 2022 Boeing Starts New A-10 Wing Deliveries to U.S. Air Force
May 25, 2022 Boeing and Ethiopian Airlines Announce Order for Five 777 Freighters
May 23, 2022 Boeing President and CEO to Speak at Bernstein Strategic Decisions Conference June 3
May 20, 2022 Boeing’s Starliner Spacecraft Completes Successful Docking to Space Station
May 20, 2022 Boeing Business Jets Names Pearson to Lead VIP Jet Sales
May 19, 2022 Boeing Starliner En Route to International Space Station
May 19, 2022 International Airlines Group Finalizes Agreement for Up to 150 737 Jets
Oct 28, 2021 Boeing Delivers First KC-46A Tanker to Japan
Oct 28, 2021 Boeing Breaks Ground for New Maintenance, Repair & Overhaul Facility in Jacksonville
Oct 27, 2021 Boeing CEO Updates Employees on Third-Quart

In [6]:
temp_df = pd.DataFrame(article_list,columns=['date','title','article','url','thumbnail','author'])
temp_df.head()

,date,title,article,url,thumbnail,author
0,"Jun 1, 2022",Boeing Teams with Canadian Industry to Offer P...,"OTTAWA, Ontario, June 1, 2022 – Boeing [NYSE: ...",https://boeing.mediaroom.com/news-releases-sta...,https://boeing.mediaroom.com/file.php/99173/32...,Boeing
1,"Jun 1, 2022",Boeing Statement on Germany’s STH Heavy-Lift H...,"BERLIN, June 1, 2022 — The German Government t...",https://boeing.mediaroom.com/news-releases-sta...,https://boeing.mediaroom.com/file.php/99170/32...,Boeing
2,"May 25, 2022","Boeing, NASA Complete First Starliner Space St...","WHITE SANDS, NEW MEXICO, May 25, 2022 — Boeing...",https://boeing.mediaroom.com/news-releases-sta...,https://boeing.mediaroom.com/file.php/99100/St...,Boeing
3,"May 25, 2022",Boeing Starts New A-10 Wing Deliveries to U.S....,"OGDEN, Utah, May 25, 2022 — Boeing [NYSE: BA],...",https://boeing.mediaroom.com/news-releases-sta...,https://upload.wikimedia.org/wikipedia/commons...,Boeing
4,"May 25, 2022",Boeing and Ethiopian Airlines Announce Order f...,", /PRNewswire/ -- Boeing [NYSE:BA] and Ethiopi...",https://boeing.mediaroom.com/2022-05-25-Boeing...,https://boeing.mediaroom.comhttps://mma.prnews...,Boeing


In [7]:
def see_data(iloc_no=random.randint(0,len(temp_df))):
    print(temp_df.iloc[iloc_no]['date'],temp_df.iloc[iloc_no]['title'])
    print(f"\n{temp_df.iloc[iloc_no]['author']} {temp_df.iloc[iloc_no]['url']}")
    print(f"\n{temp_df.iloc[iloc_no]['article']}")
    print(f"\n{temp_df.iloc[iloc_no]['thumbnail']}")

see_data()

Jun 1, 2022 Boeing Statement on Germany’s STH Heavy-Lift Helicopter Program

Boeing https://boeing.mediaroom.com/news-releases-statements?item=131040

BERLIN, June 1, 2022 — The German Government today announced that Boeing’s [NYSE: BA] CH-47F Chinook has been selected for its heavy-lift helicopter requirements (STH) for the German Bundeswehr.  “Boeing is honored the German government has selected the CH-47F Chinook for its STH heavy-lift helicopter requirements. With the Chinook, Germany will operate the most affordable, proven and NATO interoperable heavy-lift helicopter. We look forward to working with the U.S. and German governments to finalize this sale under the Foreign Military Sales process. Together with our Chinook Deutschland Team --- AERO-Bildung, Airbus Helicopters, CAE, ESG, Honeywell, Lufthansa Technik and Rolls-Royce Deutschland --- we are committed to delivering maximum operational availability to the German Bundeswehr for decades to come.”

https://boeing.mediaroom.co

In [8]:
#to csv
temp_df.to_csv(f'{SITE_NAME} news.csv',index = False)

#to json
temp_df.to_json(f'{SITE_NAME} news.json')

In [9]:
#to get rid of unwanteed trash created by the model use 
try:
    os.remove("temp.jpg")
    os.remove("temp.pdf")
except:
    print("No trash found")

No trash found
